# Setup

In [1]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import build_vocab_from_iterator
import torch.nn.functional as F


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.preprocessing import LabelEncoder
nltk.download('punkt')

from tqdm.auto import tqdm

RANDOM_SEED = 42


# Set the seed value all over the place to make this reproducible.
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


device(type='cuda', index=0)

# Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
cd drive/MyDrive/datasets

[Errno 2] No such file or directory: 'drive/MyDrive/datasets'
/content


# Функция обучения

In [3]:
def train(model, optimizer, criterion, n_epochs, train_loader, test_loader):

  loss_train = []
  accuracy_train = []

  model.to(device)

  for epoch in range(n_epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f"Training epoch {epoch + 1}/{n_epochs}"):
        inputs = batch["X"].to(device)
        labels = batch["y"].to(device)
        output = model(inputs)

        loss = criterion(output, labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
      for batch in tqdm(test_loader, desc=f"Testing epoch {epoch + 1}/{n_epochs}"):
        inputs = batch["X"].to(device)
        labels = batch["y"].to(device)
        output = model(inputs)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
    accuracy_train.append(test_accuracy)
    print('Epoch [{}/{}], Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(epoch + 1, n_epochs, loss.item(), test_accuracy * 100))
    loss_train.append(loss.item())


# 1. Генерирование русских имен при помощи RNN

Датасет: https://disk.yandex.ru/i/2yt18jHUgVEoIw

1.1 На основе файла name_rus.txt создайте датасет.
  * Учтите, что имена могут иметь различную длину
  * Добавьте 4 специальных токена:
    * `<PAD>` для дополнения последовательности до нужной длины;
    * `<UNK>` для корректной обработки ранее не встречавшихся токенов;
    * `<SOS>` для обозначения начала последовательности;
    * `<EOS>` для обозначения конца последовательности.
  * Преобразовывайте строку в последовательность индексов с учетом следующих замечаний:
    * в начало последовательности добавьте токен `<SOS>`;
    * в конец последовательности добавьте токен `<EOS>` и, при необходимости, несколько токенов `<PAD>`;
  * `Dataset.__get_item__` возращает две последовательности: последовательность для обучения и правильный ответ.
  
  Пример:
  ```
  s = 'The cat sat on the mat'
  # преобразуем в индексы
  s_idx = [2, 5, 1, 2, 8, 4, 7, 3, 0, 0]
  # получаем x и y (__getitem__)
  x = [2, 5, 1, 2, 8, 4, 7, 3, 0]
  y = [5, 1, 2, 8, 4, 7, 3, 0, 0]
  ```

### Read file

In [4]:
with open('/content/drive/MyDrive/DL_FU/name_rus.txt', 'r',  encoding = 'cp1251') as file:
  lines = file.readlines()

In [5]:
lines = [line.replace('\n', '' ) for line in lines]

In [6]:
lines

['авдокея',
 'авдоким',
 'авдоня',
 'авдотька',
 'авдотьюшка',
 'авдотья',
 'авдоха',
 'авдоша',
 'авдуля',
 'авдуся',
 'аграфена',
 'аграфенка',
 'агрипинка',
 'агрипка',
 'агриппа',
 'агриппина',
 'агриппич',
 'агриппична',
 'ада',
 'адам',
 'адамушка',
 'адамыч',
 'адаша',
 'аделина',
 'адя',
 'аида',
 'аидка',
 'акилина',
 'акилинка',
 'аксиньюшка',
 'аксинья',
 'аксюня',
 'аксюта',
 'аксюха',
 'аксюша',
 'акся',
 'акулина',
 'акулинка',
 'акуля',
 'ала',
 'албин',
 'албина',
 'алевтин',
 'алевтина',
 'алевтинка',
 'алевтиныч',
 'алека',
 'алексана',
 'александр',
 'александра',
 'александрин',
 'александрина',
 'александринка',
 'александрушка',
 'александрыч',
 'алексаня',
 'алексаха',
 'алексаша',
 'алексеич',
 'алексей',
 'алексейка',
 'алексий',
 'алексина',
 'алексинка',
 'алексюха',
 'алексюша',
 'алекся',
 'алена',
 'аленка',
 'аленя',
 'алеха',
 'алеша',
 'алина',
 'алинка',
 'алиса',
 'алиска',
 'алла',
 'альба',
 'альбин',
 'альбина',
 'альбинка',
 'альбиныч',
 'алюня',


### Vocab

In [7]:
vocab = build_vocab_from_iterator(lines, specials=["<PAD>", "<UNK>", "<SOS>", "<EOS>"])
token2inx = vocab.get_stoi()
inx2token = {inx: token for token, inx in vocab.get_stoi().items()}

In [8]:
token2inx

{'ж': 32,
 'э': 31,
 'з': 30,
 'б': 29,
 'к': 11,
 'е': 9,
 'о': 14,
 'н': 5,
 'а': 4,
 'в': 16,
 'ы': 28,
 'и': 6,
 'х': 21,
 '<SOS>': 2,
 '<EOS>': 3,
 '<UNK>': 1,
 'я': 8,
 'л': 7,
 '<PAD>': 0,
 'ш': 19,
 'с': 13,
 'м': 15,
 'ю': 17,
 'у': 18,
 'д': 20,
 'г': 22,
 'т': 12,
 'п': 23,
 'ч': 24,
 'й': 26,
 'ц': 33,
 'ь': 25,
 'р': 10,
 'ф': 27}

### X и y

#### Sample

In [41]:
# maxlen = 40
# step = 3
# sentences = []
# next_chars = []
# for i in range(0, len(text_nietzsche) - maxlen, step):
#     sentences.append(text_nietzsche[i: i + maxlen])
#     next_chars.append(text_nietzsche[i + maxlen])
# print('nb sequences:', len(sentences))

# print('Vectorization...')
# x = np.zeros((len(sentences), maxlen, len(token2inx)))
# y = np.zeros((len(sentences), len(token2inx)))
# for i, sentence in enumerate(sentences):
#     for t, char in enumerate(sentence):
#         x[i, t, token2inx[char]] = 1
#     y[i, token2inx[next_chars[i]]] = 1


maxlen = 3
step = 1
X = []
y = []

for name in lines:
  print(name)
  name1 = [token2inx.get(char,1) for char in name]
  name1.insert(0, token2inx["<SOS>"])
  name1.append(token2inx["<EOS>"])
  for i in range(0, len(name) - maxlen, step):
    print(name[i: i + maxlen], name1[i: i + maxlen])
    print(name[i + maxlen], name1[i + maxlen])
    X.append(name1[i: i + maxlen])
    y.append(name1[i + maxlen])
  break


авдокея
авд [2, 4, 16]
о 20
вдо [4, 16, 20]
к 14
док [16, 20, 14]
е 11
оке [20, 14, 11]
я 9


#### Vector

In [9]:
maxlen = 3
step = 1
X = []
y = []

for name in lines:
  name = [token2inx.get(char,1) for char in name]
  name.insert(0, token2inx["<SOS>"])
  name.append(token2inx["<EOS>"])
  for i in range(0, len(name) - maxlen, step):
    X.append(name[i: i + maxlen])
    y.append(name[i + maxlen])

X = np.array(X)
y = np.array(y)

In [10]:
X

array([[ 2,  4, 16],
       [ 4, 16, 20],
       [16, 20, 14],
       ...,
       [16, 11,  4],
       [ 2,  8, 19],
       [ 8, 19,  4]])

In [11]:
y

array([20, 14, 11, ...,  3,  4,  3])

### Train and test

In [12]:
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = RANDOM_SEED)

### class Dataset

In [13]:
class NameDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y


  def __len__(self):
    return len(self.X)

  def __getitem__(self, item):
    return {"X" : torch.tensor(self.X[item]).to(torch.long) ,
            "y" : torch.tensor(self.y[item]).to(torch.long)}

In [14]:
train_dataset = NameDataset(X_train, y_train)
test_dataset = NameDataset(X_test, y_test)

### class DataLoader

In [16]:
BATCH_SIZE = 64
train_data_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE)
test_data_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE)

1.2 Создайте и обучите модель для генерации фамилии.

  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`;
  * Используйте рекуррентные слои;
  * Задача ставится как предсказание следующего токена в каждом примере из пакета для каждого момента времени. Т.е. в данный момент времени по текущей подстроке предсказывает следующий символ для данной строки (задача классификации);
  * Примерная схема реализации метода `forward`:
  ```
    input_X: [batch_size x seq_len] -> nn.Embedding -> emb_X: [batch_size x seq_len x embedding_size]
    emb_X: [batch_size x seq_len x embedding_size] -> nn.RNN -> output: [batch_size x seq_len x hidden_size]
    output: [batch_size x seq_len x hidden_size] -> torch.Tensor.reshape -> output: [batch_size * seq_len x hidden_size]
    output: [batch_size * seq_len x hidden_size] -> nn.Linear -> output: [batch_size * seq_len x vocab_size]
  ```


### Model

In [300]:
class RNNGeneratorNames(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_size):
      super(RNNGeneratorNames, self).__init__()

      self.hidden = None

      self.embedding = nn.Embedding(vocab_size, embedding_dim)
      self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True)
      self.fc = nn.Linear(hidden_size,vocab_size)

      self.dropout = nn.Dropout(p = 0.25)


  def forward(self, input_X):
    batch_size = input_X.shape[0]
    seq_len = input_X.shape[1]

    self.hidden = torch.zeros(1, batch_size, self.rnn.hidden_size) # layer, batch_size, hidden_size

    emb_X = self.embedding(input_X)
    output, self.hidden = self.rnn(emb_X, self.hidden)

    output = self.dropout(output)
    output = F.relu(output)


    output = output.reshape(batch_size * seq_len, hidden_size)
    output = self.fc(output)
    output = output.reshape(batch_size, seq_len, -1)[:, -1, :] ### last char
    return output

In [302]:
seq_len = 3
embedding_dim = 6
hidden_size = 100
EPOCHS = 10


optimizer = nn.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()


In [303]:
train(modelNameGeneration, optimizer, criterion, EPOCHS, train_data_loader, test_data_loader)

Training epoch 1/20:   0%|          | 0/128 [00:00<?, ?it/s]

Testing epoch 1/20:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch [1/20], Loss: 2.5335, Test Accuracy: 28.27%
<SOS>удикааыа<EOS>


Training epoch 2/20:   0%|          | 0/128 [00:00<?, ?it/s]

Testing epoch 2/20:   0%|          | 0/32 [00:00<?, ?it/s]

...

Testing epoch 19/20:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch [19/20], Loss: 1.6467, Test Accuracy: 51.46%
<SOS>емекслич<EOS>


Training epoch 20/20:   0%|          | 0/128 [00:00<?, ?it/s]

Testing epoch 20/20:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch [20/20], Loss: 1.6109, Test Accuracy: 51.86%
<SOS>уля<EOS>


1.3 Напишите функцию, которая генерирует фамилию при помощи обученной модели:
  * Построение начинается с последовательности единичной длины, состоящей из индекса токена `<SOS>`;
  * Начальное скрытое состояние RNN `h_t = None`;
  * В результате прогона последнего токена из построенной последовательности через модель получаете новое скрытое состояние `h_t` и распределение над всеми токенами из словаря;
  * Выбираете 1 токен пропорционально вероятности и добавляете его в последовательность (можно воспользоваться `torch.multinomial`);
  * Повторяете эти действия до тех пор, пока не сгенерирован токен `<EOS>` или не превышена максимальная длина последовательности.

При обучении каждые `k` эпох генерируйте несколько фамилий и выводите их на экран.

In [304]:
def generateName(model, max_length = 15):
  model.eval()

  name = torch.tensor([token2inx["<SOS>"]]).to(torch.long)

  while name[-1].item() != token2inx["<EOS>"] and len(name) < max_length:
    output = model(name.unsqueeze(0))
    probabilities = F.softmax(output.squeeze(), dim=0)
    next_token = torch.multinomial(probabilities, 1) # генерации случайных сэмплов в соответствии с мультиномиальным распределением.
    name = torch.cat((name, next_token), dim=0)

  str_name = "".join([inx2token[char.item()] for char in name])
  return str_name



In [307]:
generateName(modelNameGeneration, 10)

'<SOS>андуня<EOS>'

# 2. Генерирование текста при помощи RNN



2.1 Скачайте из интернета какое-нибудь художественное произведение
  * Выбирайте достаточно крупное произведение, чтобы модель лучше обучалась;

### Read file

In [4]:
with open('/content/drive/MyDrive/NLP_FU/dostoevsky.txt', 'r') as file:
  lines = file.readlines()

In [5]:
lines[:10]

['\ufeffФедор Михайлович Достоевский\n',
 'Бедные люди\n',
 'Ох уж эти мне сказочники! Нет чтобы написать что-нибудь полезное, приятное, усладительное, а то всю подноготную в земле вырывают!.. Вот уж запретил бы им писать! Ну, на что это похоже: читаешь… невольно задумываешься,\xa0— а там всякая дребедень и пойдет в голову; право бы, запретил им писать; так-таки просто вовсе бы запретил.\n',
 'Кн. В. Ф. Одоевский\n',
 'Апреля 8-го\n',
 'Бесценная моя Варвара Алексеевна!\n',
 'Вчера я был счастлив, чрезмерно счастлив, донельзя счастлив! Вы хоть раз в жизни, упрямица, меня послушались. Вечером, часов в восемь, просыпаюсь (вы знаете, маточка, что я часочек-другой люблю поспать после должности), свечку достал, приготовляю бумаги, чиню перо, вдруг, невзначай, подымаю глаза,\xa0— право, у меня сердце вот так и запрыгало! Так вы-таки поняли, чего мне хотелось, чего сердчишку моему хотелось! Вижу, уголочек занавески у окна вашего загнут и прицеплен к горшку с бальзамином, точнехонько так, как 

In [6]:
lines = [re.sub(r'[^а-яА-ЯёЁ\s\.,;:!?]', '', line.lower().strip().replace("\n", "").replace("\xa0", "")) for line in lines]
lines = " ".join(lines)
lines[:100]

'федор михайлович достоевский бедные люди ох уж эти мне сказочники! нет чтобы написать чтонибудь поле'

In [7]:
lines = lines[:1_000_000]
len(lines)

1000000

2.2 На основе выбранного произведения создайте датасет.

Отличия от задачи 1:
  * Токены <SOS>, `<EOS>` и `<UNK>` можно не добавлять;
  * При создании датасета текст необходимо предварительно разбить на части. Выберите желаемую длину последовательности `seq_len` и разбейте текст на построки длины `seq_len` (можно без перекрытия, можно с небольшим перекрытием).

### Vocab

In [8]:
vocab = build_vocab_from_iterator(lines, specials=["<PAD>", "<UNK>", "<SOS>", "<EOS>"])
token2inx = vocab.get_stoi()
inx2token = {inx: token for token, inx in vocab.get_stoi().items()}

In [9]:
token2inx

{'ъ': 43,
 'ё': 42,
 ':': 41,
 '?': 39,
 ';': 38,
 '!': 37,
 'щ': 36,
 'э': 35,
 'ю': 33,
 'х': 32,
 'ш': 31,
 'й': 30,
 'ж': 29,
 'в': 12,
 'а': 7,
 'ы': 26,
 'н': 9,
 'с': 11,
 'о': 5,
 'е': 6,
 'к': 16,
 ',': 18,
 'з': 27,
 'и': 10,
 '<SOS>': 2,
 '<EOS>': 3,
 '<UNK>': 1,
 ' ': 4,
 '<PAD>': 0,
 'я': 21,
 'л': 13,
 'ф': 40,
 'р': 14,
 'м': 15,
 'д': 17,
 'у': 19,
 'т': 8,
 'п': 20,
 'ц': 34,
 'ь': 22,
 'ч': 23,
 'г': 24,
 'б': 25,
 '.': 28}

### X и y

In [10]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(lines) - maxlen, step):
    sentences.append(lines[i: i + maxlen])
    next_chars.append(lines[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(token2inx)))
y = np.zeros((len(sentences), len(token2inx)))
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, token2inx[char]] = 1
    y[i, token2inx[next_chars[i]]] = 1

nb sequences: 333320
Vectorization...



2.3 Создайте и обучите модель для генерации текста
  * Задача ставится точно так же как в 1.2;
  * При необходимости можете применить:
    * двухуровневые рекуррентные слои (`num_layers`=2)
    * [обрезку градиентов](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)

### Train and test

In [11]:
X_train, X_test , y_train, y_test = train_test_split(x,y, test_size = 0.2)
len(X_train) , len(X_test) , len(y_train) , len(y_test)

(266656, 66664, 266656, 66664)

### class Dataset

In [12]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input = torch.Tensor(self.data[idx]).to(device)
        label = torch.Tensor(self.labels[idx]).to(device)
        sample = {'X': input, 'y': label}
        return sample

In [13]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

### class DataLoader

In [14]:
BATCH_SIZE = 256

train_data_loader = DataLoader(train_dataset,  batch_size=BATCH_SIZE)
test_data_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

### Model

In [15]:
class AnpilovGpt(nn.Module):
    def __init__(self, vocab_size, hidden_size1, hidden_size2 ):
        super().__init__()
        self.lstm = nn.LSTM(vocab_size, hidden_size1,  batch_first=True) # (batch_size, seq_len, input_size)
        self.linear = nn.Linear(in_features = hidden_size1 , out_features = hidden_size2 )
        self.projection = nn.Linear(in_features = hidden_size2 , out_features = vocab_size) # vocab_size = num_classes

        self.dropout = nn.Dropout(p = 0.25) # defult p = 0.5
        self.tanh = nn.Tanh() # against vanish gradient

    def forward(self, input):
        output, _ = self.lstm(input)  # [batch_size, seq_len, hidden_dim]
        output = output[:, -1]
        output = self.dropout(self.linear(self.tanh(output)))  # [batch_size, seq_len, hidden_dim]
        projection = self.projection(self.tanh(output))
        return projection

In [16]:
n_epochs = 20
vocab_size = len(vocab)
hidden_size1 = 256
hidden_size2 = 128


model = AnpilovGpt(vocab_size, hidden_size1, hidden_size2).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [22]:
loss_train = []
accuracy_train = []

for epoch in range(n_epochs):
    model.train()
    for batch in tqdm(train_data_loader , desc=f'Training epoch {epoch + 1}:'):
        inputs = batch['X']
        labels = batch['y']
        output = model(inputs)
        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    model.eval()
    total = 0
    correct = 0
    for batch in tqdm(test_data_loader , desc=f'Testing epoch {epoch + 1}:'):
          inputs = batch['X']
          labels = batch['y']
          output = model(inputs)
          _, predicted = torch.max(output.data, 1)
          labels = torch.nonzero(labels == 1)[:,1]

          total += labels.size(0)
          correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
    accuracy_train.append(test_accuracy)

    print('Epoch [{}/{}], Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(epoch + 1, n_epochs, loss.item(), test_accuracy * 100))
    loss_train.append(loss.item())
    generate_text(model, "я люблю ", num = 100)


Training epoch 1::   0%|          | 0/1042 [00:00<?, ?it/s]

Testing epoch 1::   0%|          | 0/261 [00:00<?, ?it/s]

Epoch [1/20], Loss: 2.1674, Test Accuracy: 40.65%
я люблю не после не после не после не после не после не после не после не после не после не после не после н


Training epoch 2::   0%|          | 0/1042 [00:00<?, ?it/s]

Testing epoch 2::   0%|          | 0/261 [00:00<?, ?it/s]

...

Testing epoch 19::   0%|          | 0/261 [00:00<?, ?it/s]

Epoch [19/20], Loss: 1.0106, Test Accuracy: 48.14%
я люблю и все это своей уже надо бывал мне со степаным трофимства и почти в правлекие и теперь не могу они в


Training epoch 20::   0%|          | 0/1042 [00:00<?, ?it/s]

Testing epoch 20::   0%|          | 0/261 [00:00<?, ?it/s]

Epoch [20/20], Loss: 0.9681, Test Accuracy: 48.04%
я люблю и не все это завести после соботанности полезный произрет прежде всегдашавно примералось, что она не



2.4 Напишите функцию, которая генерирует фрагмент текста при помощи обученной модели
  * Процесс генерации начинается с небольшого фрагмента текста `prime`, выбранного вами (1-2 слова)
  * Сначала вы пропускаете через модель токены из `prime` и генерируете на их основе скрытое состояние рекуррентного слоя `h_t`;
  * После этого вы генерируете строку нужной длины аналогично 1.3

In [18]:
def generate_text(model, text, num = 100):
      model.eval()
      for i in range(num):
        x = torch.zeros(len(text), vocab_size).to(device)
        for j, char in enumerate(text):
          x[j, token2inx[char]] = 1
        x = x.unsqueeze(0)


        output = model(x)
        _, predicted = torch.max(output.data, 1)

        text += inx2token[int(predicted)]
      print(text)

In [24]:
generate_text(model, "я люблю ", num = 100)

я люблю и не все это завести после соботанности полезный произрет прежде всегдашавно примералось, что она не
